In [2]:
import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning)
import requests
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import random
import numpy as np
import pandas as pd
import itertools
from sklearn import svm
from sklearn import tree
from urllib import request
from random import randint
#from wordcloud import WordCloud
from nltk import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt
import gensim
from gensim.models import LdaModel, Word2Vec
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from nltk.corpus import stopwords
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer
#!pip install mlxtend
#from mlxtend.evaluate import bias_variance_decomp
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hadyo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hadyo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hadyo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
books_url_list = ["https://www.gutenberg.org/files/19719/19719-0.txt",
                  "https://www.gutenberg.org/cache/epub/28434/pg28434.txt",
                  "https://www.gutenberg.org/cache/epub/15147/pg15147.txt",
                  "https://www.gutenberg.org/cache/epub/17866/pg17866.txt",
                  "https://www.gutenberg.org/files/3772/3772-0.txt"]

# Preprocessing

In [4]:
#function read raw data
def read_books(url):
    files = []
    for i in url:
        file = requests.get(url = i)
        files.append(file.content.decode('utf-8'))
    return files

#read raw data
raw_dataset = read_books(books_url_list)
len(raw_dataset)

5

In [5]:
#Function cleaned tokens
def cleaned_text(raw_text):
    text_beg = raw_text.find("*** START OF")
    text_end = raw_text.find("*** END OF")

    raw_text = raw_text[text_beg : text_end]
    raw_text = re.findall(r"[a-zA-Z]+", raw_text)
    raw_text = " ".join(raw_text).lower()
    tokens = word_tokenize(raw_text)
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return tokens
#Getting cleaned tokens
tokens = []
for i in range(len(raw_dataset)):
    tokens.append(cleaned_text(raw_dataset[i]))
len(tokens)

5

In [6]:
#function to extract author name, book name and labels
book_author_mapping = {}
def extract_book_author(raw_text, index):
    label = chr(index + 97)
        
    title_beg = raw_text.find("Title") + 7
    title_end = raw_text.find("\r\n\r\nAuthor")
    title = raw_text[title_beg : title_end]

    author_beg = raw_text.find("Author") + 8
    c = 0
    while raw_text[author_beg + c] != "\r":
        c += 1
        author_end = author_beg + c
    author = raw_text[author_beg : author_end]
    
    book_author_mapping[label] = [title, author]
    
    return label, title, author

#getting the author name, book name, labels
label, author, title = [], [], []
for i in range(len(raw_dataset)):
    label_temp, title_temp, author_temp = extract_book_author(raw_dataset[i],i)
    label.append(label_temp)
    author.append(author_temp)
    title.append(title_temp)
    
print(len(label))
print(len(author))
print(len(title))

5
5
5


In [7]:
#function to partition data
def partitioned_text(tokens, title, author, label):
    final_book_partitions = []
    book_partitions = []
    for j in range(len(tokens)):
        subtractor = len(tokens[j])
        for i in range(0, 200):
            if(subtractor > 150):
                partition = [" ".join(tokens[j][i : i + 150]), title[j], author[j], label[j]]
                book_partitions.append(partition)
                subtractor -= 150
    return book_partitions

In [8]:
#getting partitoins
partitions = pd.DataFrame(partitioned_text(tokens, title, author, label),columns = ['partitions', 'title', 'author', 'label'])

partitions = shuffle(partitions)
partitions

,partitions,title,author,label
604,murder gunroom produced greg week mary meehan ...,Murder in the Gunroom,Henry Beam Piper,d
327,glorious work parent good almighty thine unive...,The Astronomy of Milton's 'Paradise Lost',Thomas Orchard,b
486,czechoslovakia samuel butler preface art needl...,Beeton's Book of Needlework,Isabella Beeton,c
409,barkley pg online distributed proofreading tea...,Beeton's Book of Needlework,Isabella Beeton,c
771,suspected would make satisfying lover one migh...,Murder in the Gunroom,Henry Beam Piper,d
...,...,...,...,...
764,serviceable friend particularly unpleasant ene...,Murder in the Gunroom,Henry Beam Piper,d
625,york alfred knopf first edition colonel henry ...,Murder in the Gunroom,Henry Beam Piper,d
581,berlin wool work point lace guipure art prefix...,Beeton's Book of Needlework,Isabella Beeton,c
286,p consequenc changed consequence p geographiea...,The Astronomy of Milton's 'Paradise Lost',Thomas Orchard,b
